In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import tkinter as tk
from tkinter import filedialog
import os
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

In [3]:
# --- UI要素の作成 ---
style = {'description_width': 'initial'}
run_button_khy = widgets.Button(description="ファイルを選択してkhyを解析", button_style='success', layout=widgets.Layout(width='300px'))
output_area_khy = widgets.Output()

# ↓↓↓ 横軸の最大値を設定するためのウィジェットを追加 ↓↓↓
xmax_widget = widgets.IntText(description='グラフ横軸の最大値 (分):', value=400, style=style)

# --- UIの表示 ---
# フォームの表示部分をウィジェットの名前に変更
form_khy = widgets.VBox([
    widgets.HTML(value="<h3>khy 解析</h3>"),
    xmax_widget, # ← 追加
    run_button_khy,
    output_area_khy
])

display(form_khy)
# --- モデル関数定義 (変更なし) ---
def exponential_decay(t_sec, i0, k_hy):
    return i0 * np.exp(-k_hy * t_sec)

# --- メインの処理関数 ---
def on_khy_run_button_clicked(b):
    with output_area_khy:
        clear_output(wait=True)
        
        # --- ファイル選択とパラメータ取得 ---
        root = tk.Tk()
        root.withdraw()
        root.attributes('-topmost', True)
        file_paths = filedialog.askopenfilenames(
            title="khy解析用のCSVファイルを選択 (複数選択可)",
            filetypes=[("CSV and Excel files", "*.xlsx *.csv"), ("All files", "*.*")]
        )
        if not file_paths:
            print("ファイルが選択されませんでした。")
            return
            
        xmax_minutes = xmax_widget.value # 先に横軸の最大値を取得

        # --- グラフの準備 ---
        plt.figure(figsize=(10, 6))
        print("--- khy 解析結果 ---")

        # --- 各ファイルに対してループ処理 ---
        for path in file_paths:
            try:
                if path.endswith('.csv'): df = pd.read_csv(path)
                else: df = pd.read_excel(path)
                
                filename = os.path.basename(path)
                print(f"\nProcessing: {filename}")

                required_columns = ['time_min', 'gtp_intensity']
                if not all(col in df.columns for col in required_columns):
                    print(f"  -> 警告: '{filename}' に必要な列 ('time_min', 'gtp_intensity') がありません。スキップします。")
                    continue
                    
                df_gtp = df[required_columns].dropna()
                if df_gtp.empty or len(df_gtp) < 2:
                    print(f"  -> 警告: '{filename}' にフィッティング可能なデータがありません。スキップします。")
                    continue
                
                df_gtp['time_sec'] = df_gtp['time_min'] * 60
                initial_intensity = df_gtp['gtp_intensity'].iloc[0]
                df_gtp['gtp_ratio'] = df_gtp['gtp_intensity'] / initial_intensity

                popt, pcov = curve_fit(exponential_decay, df_gtp['time_sec'], df_gtp['gtp_ratio'], p0=[1.0, 0.0001])
                i0_fit, k_hy_fit = popt
                k_hy_err = np.sqrt(np.diag(pcov))[1]
                print(f"  k_hy = {k_hy_fit:.2E} ± {k_hy_err:.1E} (s⁻¹)")

                # --- グラフへの描画 ---
                plt.scatter(df_gtp['time_min'], df_gtp['gtp_ratio'], label=f'{filename}')
                
                # ↓↓↓ フィット曲線用の時間軸を、0分から指定した最大値までで作成するよう修正 ↓↓↓
                time_fit_min = np.linspace(0, xmax_minutes, 400) # 0から最大値まで400点で生成
                time_fit_sec = time_fit_min * 60
                
                fit_label = (
                    r'$[Ras_{GTP}]_t = [Ras_{GTP}]_0 \cdot e^{-k_{hy} \cdot t}$' + '\n'
                    f'$k_{{hy}} = ${k_hy_fit:.2E} s⁻¹'
                )
                plt.plot(time_fit_min, exponential_decay(time_fit_sec, *popt), linestyle='--', label=fit_label)

            except Exception as e:
                print(f"ファイル '{os.path.basename(path)}' の処理中にエラーが発生しました: {e}")
        
        print("--------------------")
        
        # --- グラフの最終仕上げ ---
        plt.title('GTP Hydrolysis Kinetics (khy)')
        plt.xlabel('Time (min)')
        plt.ylabel('GTP bound ratio')
        plt.legend(fontsize='small')
        plt.grid(True)
        
        # 横軸の表示範囲を設定
        plt.xlim(-5, xmax_minutes)
        plt.ylim(-0.1, 1.1)
        
        plt.show()

# --- ボタンと関数を紐付け ---
run_button_khy.on_click(on_khy_run_button_clicked)